In [1]:
import numpy as np
import pandas as pd
import uuid
import os
import time
import json

class Provenance:
    
    # Constants:
    NAMESPACE_FUNC = 'activity:'
    NAMESPACE_ENTITY = 'entity:'
    INPUT = 'input'
    OUTPUT = 'output'
    
    CHUNK_SIZE = 60000
    
    # PROV-N objects
    ENTITY = 'prov:entity'
    GENERATED_ENTITY = 'prov:generatedEntity'
    USED_ENTITY = 'prov:usedEntity'
    ACTIVITY = 'prov:activity'
    
    # PROV-N relations
    GENERATION = 'wasGeneratedBy'
    USE = 'used'
    DERIVATION = 'wasDerivedFrom'
    INVALIDATION = 'wasInvalidatedBy'
    
    def __init__(self, df, results_path=None):
        # Inizialize provenance activities, relations and new entities
        self.current_act = []
        self.current_relations = []
        self.new_entities = []
        
         # Set input dataframe parameters:
        self.current_m, self.current_n = df.shape
        self.current_columns = df.columns
        self.current_index = df.index
        
        # Create provenance entities of the input dataframe:
        self.current_ent = self.create_prov_entities(df, self.INPUT)
        
        # Initialize operation number:
        self.operation_number = -1
        self.instance = self.OUTPUT + str(self.operation_number)
        
        # Set results path:
        self.results_path = 'results/' + time.strftime('%Y%m%d-%H%M%S') if results_path is None else results_path
        
        # Save input provenance document
        self.save_json_prov(os.path.join(self.results_path, self.INPUT))
        
    def create_entity(self, ent_id, record_id, value, feature_name, instance):
        """Create a provenance entity.
        Return a dictionary with the id and the attributes of the entity."""
        # Get attributes:
        other_attributes = {}
        other_attributes['record_id'] = record_id
        other_attributes['value'] = value
        other_attributes['feature_name'] = feature_name
        #other_attributes['instance'] = instance
        
        # Add entity to new numpy array entities:
        ent = {'identifier': ent_id, 'attributes': other_attributes}
        self.new_entities.append(ent)
        
        return ent
    
    def create_activity(self, function_name, features_name=None, other_attributes=None):
        """Create a provenance activity and add to the current activities array.
        Return the id of the new prov activity."""
        # Get default activity attributes:
        attributes = {}
        attributes['function_name'] = function_name
        if features_name is not None:
            attributes['features_name'] =  features_name
        attributes['operation_number'] = str(self.operation_number)
        
        # Join default and extra attributes:
        if other_attributes is not None:
            attributes.update(other_attributes)
            
        act_id = self.NAMESPACE_FUNC + str(uuid.uuid4())
        
        # Add activity to current provenance document:
        act = {'identifier': act_id, 'attributes': attributes}
        self.current_act.append(act)
        
        return act_id
    
    def create_relation(self, relation_type, **relation):
        """Add a relation to the current relations array.
        Return the new relation."""
        if relation_type == self.GENERATION:
            relation[self.ENTITY] = relation.pop('a')
            relation[self.ACTIVITY] = relation.pop('b')
        elif relation_type == self.USE:
            relation[self.ACTIVITY] = relation.pop('a')
            relation[self.ENTITY] = relation.pop('b')
        elif relation_type == self.DERIVATION:
            relation[self.GENERATED_ENTITY] = relation.pop('a')
            relation[self.USED_ENTITY] = relation.pop('b')
        elif relation_type == self.INVALIDATION:
            relation[self.ENTITY] = relation.pop('a')
            relation[self.ACTIVITY] = relation.pop('b')
        #else:
            #TODO: Exception, invalid relation
            
        relation.update({'prov:relation_type':relation_type})
        self.current_relations.append(relation)
        
        return relation
        
    def create_prov_entities(self, dataframe, instance=None):
        """Return a numpy array of new provenance entities related to the dataframe."""
        instance = self.instance if instance is None else instance
        columns = dataframe.columns
        
        # Copy input values in array
        # values = np.array(dataframe.values)
        # Create a function that adds input entities to prov document
        # createEntities = lambda i,j: self.create_entity(self.NAMESPACE_ENTITY + str(uuid.uuid4()), 
        #                                                  str(values[i][j]), 
        #                                                  columns[j], 
        #                                                  instance)
        # entities = np.fromfunction(np.vectorize(createEntities), values.shape, dtype=object)
        
        # Create output array of entities:
        entities = np.empty(dataframe.shape, dtype=object)
        for i in range(self.current_m):
            record_id = str(uuid.uuid4())
            for j in range(self.current_n):
                ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                value = str(dataframe.iat[i, j])
                 # Add entity to current provenance document:
                entities[i][j] = self.create_entity(ent_id, record_id, value, columns[j], instance)

        return entities
    
    def set_current_values(self, dataframe, entities_out):
        """Update values of current entities after every operation."""
        # Set output dataframe entities:
        self.current_m, self.current_n = dataframe.shape
        self.current_columns = dataframe.columns
        self.current_index = dataframe.index
        self.current_ent = entities_out
        
    def initialize(self):
        self.current_act = []
        self.current_relations = []
        self.new_entities = []

        # Increment operation number:
        self.operation_number += 1
        self.instance = self.OUTPUT + str(self.operation_number)
            
    def save_json_prov(self, nameFile):
        """Save provenance in json file."""
        if not os.path.exists(nameFile):
            os.makedirs(nameFile)
        
        ents_path = os.path.join(nameFile, 'entities')
        acts_path = os.path.join(nameFile, 'activities.json')
        rel_path = os.path.join(nameFile, 'relations')
                
        # Save entities: 
        #entities = list(self.current_ent.flatten())
        entities = self.new_entities
        
        if entities:
            for i in range(0, len(entities), self.CHUNK_SIZE):
                output_name = ents_path + '.json' if i//self.CHUNK_SIZE == 0 else ents_path + '_' + str(i//self.CHUNK_SIZE) + '.json'
                with open(output_name, 'w', encoding='utf-8') as ents_file:
                    ents = entities[i:i+self.CHUNK_SIZE]
                    json.dump(ents, ents_file, ensure_ascii=False, indent=4)
                
        # Save activities:
        if self.current_act:
            with open(acts_path, 'w', encoding='utf-8') as acts_file:
                json.dump(self.current_act, acts_file, ensure_ascii=False, indent=4)
                
        # Save all relations:
        if self.current_relations:
            for i in range(0, len(self.current_relations), self.CHUNK_SIZE):
                output_name = rel_path + '.json' if i//self.CHUNK_SIZE == 0 else rel_path + '_' + str(i//self.CHUNK_SIZE) + '.json'
                with open(output_name, 'w', encoding='utf-8') as rel_file:
                    rels = self.current_relations[i:i+self.CHUNK_SIZE]
                    json.dump(rels, rel_file, ensure_ascii=False, indent=4)
    
    def timing(f):
        def wrap(*args):
            # Get timing of provenance function:
            time1 = time.time()
            ret = f(*args)
            time2 = time.time()
            text = '{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0)
            print(text)
            
            self = args[0]
            path = os.path.join(self.results_path, 'log_file.txt')
            with open(path, 'a+') as log_file:
                log_file.write('[' + time.strftime("%d/%m-%H:%M:%S") +']' + text + '\n')
            
            #duration = time2 - time1
            #print(f.__name__
            #      + ' finished in ' 
            #      + time.strftime('%H:%M:%S', time.gmtime(duration)))

            return ret
        return wrap
    
        
    ###
    ###  PROVENANCE METHODS
    ###

    @timing
    def get_prov_binarizer(self, df_out, columnsName, function_name='Binarizer'): 
        """Return provenance document related to binarization function.
        
        Keyword argument:
        df_out -- the output dataframe
        columnsName -- list of binarized columns name
        """
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        
        # Output values:
        columns_out = df_out.columns
        
        for col_name in columnsName:
            act_id = self.create_activity(function_name, col_name)
            col_index = columns_out.get_loc(col_name)
            for i in range(self.current_m):
                e_in = entities_in[i][col_index]
                e_in_identifier = e_in['identifier']
                record_id = e_in['attributes']['record_id']
                value = str(df_out.iat[i, col_index])
                
                # Create a new entity with new value:
                ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                e_out = self.create_entity(ent_id, record_id, value, col_name, self.instance)
                e_out_identifier = e_out['identifier']
                    
                self.create_relation(self.GENERATION, a=e_out_identifier, b=act_id)
                self.create_relation(self.USE, a=act_id, b=e_in_identifier)
                self.create_relation(self.DERIVATION, a=e_out_identifier, b=e_in_identifier)
                    
                entities_in[i][col_index] = e_out        
                
        # Update current values:
        self.set_current_values(df_out, entities_in) 
        
        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, str(self.instance)))
        
        return self
    
    @timing
    def get_prov_feature_transformation(self, df_out, columnsName, function_name='Feature Transformation'):
        """Return provenance document related to features trasformation function.
        
        Keyword argument:
        df_out -- the output dataframe
        columnsName -- list of transformed columns name
        """
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        
        # Output values:
        columns_out = df_out.columns
        
        for col_name in columnsName:
            act_id = self.create_activity(function_name, col_name)
            col_index = columns_out.get_loc(col_name)
            for i in range(self.current_m):
                e_in = entities_in[i][col_index]
                e_in_identifier = e_in['identifier']
                record_id = e_in['attributes']['record_id']
                value = str(df_out.iat[i, col_index])
                
                # Create a new entity with new value:
                ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                e_out = self.create_entity(ent_id, record_id, value, col_name, self.instance)
                e_out_identifier = e_out['identifier']
                    
                self.create_relation(self.GENERATION, a=e_out_identifier, b=act_id)
                self.create_relation(self.USE, a=act_id, b=e_in_identifier)
                self.create_relation(self.DERIVATION, a=e_out_identifier, b=e_in_identifier)
                    
                entities_in[i][col_index] = e_out
                
        # Update current values:
        self.set_current_values(df_out, entities_in)

        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))
        
        return self

    @timing
    def get_prov_space_transformation(self, df_out, columnsName, function_name='Space Transformation'):
        """Return provenance document related to space trasformation function.
        
        Keyword argument:
        df_out -- the output dataframe
        columnsName -- list of columns name joined to create the new column
        """
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        m, n = self.current_m, self.current_n

        # Output values:
        m_new, n_new = df_out.shape
        columns_out = df_out.columns
        
        # Create entities of the output dataframe:
        entities_out = np.empty(df_out.shape, dtype=object)
        new_entities = np.empty((m, n_new-n), dtype=object)
        
        # Get feature indexes used for space transformation:
        indexes = []
        for feature in columnsName:
            indexes.append(columns_out.get_loc(feature))
            
        # Create space transformation activity:
        act_id = self.create_activity(function_name, ', '.join(columnsName))
        
        # Get provenance related to existent data:
        for i in range(m):
            for j in indexes:
                e_in = entities_in[i][j]
                ent_id = e_in['identifier']
                self.create_relation(self.USE, a=act_id, b=ent_id)
                    
        # Get provenance related to the new column:
        for i in range(m):
            first_ent = entities_in[i][0]
            record_id = first_ent['attributes']['record_id']
            for j in range(n, n_new):
                value = str(df_out.iat[i, j])
                ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                e_out = self.create_entity(ent_id, record_id, value, columns_out[j], self.instance)
                e_out_identifier = e_out['identifier']
                new_entities[i][j-n] = e_out
                self.create_relation(self.GENERATION, a=e_out_identifier, b=act_id)
                for index in indexes:
                    e_in = entities_in[i][index]
                    e_in_identifier = e_in['identifier']
                    self.create_relation(self.DERIVATION, a=e_out_identifier, b=e_in_identifier)
                    
        entities_out = np.concatenate((entities_in, new_entities), axis=1)
                
        # Update current values:
        self.set_current_values(df_out, entities_out)
        
        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))

        return self

    @timing
    def get_prov_feature_selection(self, df_out, function_name='Feature Selection'):
        """Return provenance document related to feature selection function."""
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        columns_in = self.current_columns
        m, n = self.current_m, self.current_n
        
        # Output values:
        columns_out = df_out.columns
        m_new, n_new = df_out.shape
        # Create entities of the output dataframe:
        entities_out = np.empty(df_out.shape, dtype=object)

        delColumnsName = set(columns_in) - set(columns_out)  # List of selected columns
        
        # Create feature selection activity:
        act_id = self.create_activity(function_name, ', '.join(delColumnsName))
        
        delColumns = []
        for colName in delColumnsName:
            j = columns_in.get_loc(colName)
            delColumns.append(j)
            for i in range(m):
                e_in = entities_in[i][j]
                e_in_identifier = e_in['identifier']
                self.create_relation(self.INVALIDATION, a=e_in_identifier, b=act_id)
        
        entities_out = np.delete(entities_in, delColumns, axis=1)
                    
        # Update current values:
        self.set_current_values(df_out, entities_out)
        
        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))

        return self
    
    @timing
    def get_prov_dim_reduction(self, df_out, function_name='Dimensionality reduction'):
        """Return provenance document related to selection or projection."""
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        columns_in = self.current_columns
        index_in = self.current_index
        m, n = self.current_m, self.current_n
        
        # Output values:
        columns_out = df_out.columns
        index_out = df_out.index
        m_new, n_new = df_out.shape
        # Create entities of the output dataframe:
        # entities_out = np.empty(df_out.shape, dtype=object)
        
        delColumnsName = set(columns_in) - set(columns_out) # List of deleted columns
        delIndex = set(index_in) - set(index_out) # List of deleted columns
        
        # Create selection activity:
        act_id = self.create_activity(function_name, ', '.join(delColumnsName))
        
        for i in delIndex:
            for j in range(n):
                e_in = entities_in[i][j]
                e_in_identifier = e_in['identifier']
                self.create_relation(self.INVALIDATION, a=e_in_identifier, b=act_id)
        
        delColumns = []
        for colName in delColumnsName:
            j = columns_in.get_loc(colName)
            delColumns.append(j)
            for i in range(m):
                e_in = entities_in[i][j]
                e_in_identifier = e_in['identifier']
                self.create_relation(self.INVALIDATION, a=e_in_identifier, b=act_id)
        
        entities_in = np.delete(entities_in, list(delIndex), axis=0)
        entities_out = np.delete(entities_in, delColumns, axis=1)
        
        # Update current values:
        self.set_current_values(df_out, entities_out)
        
        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))

        return self

    @timing
    def get_prov_instance_generation(self, df_out, function_name='Instance Generation'):
        """Return provenance document related to instance generation function."""
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        m, n = self.current_m, self.current_n
        
        # Output values:
        columns_out = df_out.columns
        m_new, n_new = df_out.shape
        
        # Create numpy array of new entities:
        #entities_out = np.empty(df_out.shape, dtype=object)
        new_entities = np.empty((m_new-m, n), dtype=object)
        
        acts = []
        for j in range(n):
            # Create function for every columns
            act_id = self.create_activity(function_name, columns_out[j])
            acts.append(act_id)
            
        for i in range(m_new):
            record_id = str(uuid.uuid4())
            for j in range(n):
                act_id = acts[j]
                value = str(df_out.iat[i, j])
                if i < m:
                    # Provenance of existent data
                    e_in = entities_in[i][j]
                    ent_id = e_in['identifier']
                    self.create_relation(self.USE, a=act_id, b=ent_id)
                else:
                    # Provenance of new data
                    ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                    e_out = self.create_entity(ent_id, record_id, value, columns_out[j], self.instance)
                    e_out_identifier = e_out['identifier']
                    new_entities[i-m][j] = e_out
                    self.create_relation(self.GENERATION, a=e_out_identifier, b=act_id)
                    
        entities_out = np.concatenate((entities_in, new_entities), axis=0)
        
        # Update current values:
        self.set_current_values(df_out, entities_out)
        
        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))

        return self

    @timing
    def get_prov_onehot_encode(self, df_out, onehot_cols, onehot_cols_map, function_name='OneHot Encoding'):
        """Return provenance document related to one-hot encoding function.
        
        Keyword argument:
        df_out -- the output dataframe
        onehot_cols -- list of One-Hot encoded columns 
        onehot_cols_map -- map(key, values)
                           where key is the One-Hot encoded column name
                           and values is an array of the new columns name
        """
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        columns_in = self.current_columns

        # Output values:
        m_new, n_new = df_out.shape
        columns_out = df_out.columns
        # Create entities of the output dataframe:
        entities_out = np.empty(df_out.shape, dtype=object)
        
        activities_dict = {}
        
        # Create functions (one for all one hot encoded feature):
        for col_name in onehot_cols:
            act_id = self.create_activity(function_name, col_name)
            activities_dict[col_name] = act_id
            
            # Add input entities used by functions:
            column_index = columns_in.get_loc(col_name) if col_name in columns_in else -1
            for i in range(self.current_m):
                e_in = entities_in[i][column_index]
                e_in_identifier = e_in['identifier']
                self.create_relation(self.USE, a=act_id, b=e_in_identifier)
                if col_name not in columns_out:
                    self.create_relation(self.INVALIDATION, a=e_in_identifier, b=act_id)
                
        new_columns = set(columns_out) - set(columns_in)
        
        # Add generated entities:
        for i in range(self.current_m):
            first_ent = entities_in[i][0]
            record_id = first_ent['attributes']['record_id']
            for column_out_name in new_columns:
                j = columns_out.get_loc(column_out_name)
                value = str(df_out.iat[i, j])
                for k,v in onehot_cols_map.items():
                    if column_out_name in v:
                        column_name = k
                    
                ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                e_out = self.create_entity(ent_id, record_id, value, column_out_name, self.instance)
                e_out_identifier = e_out['identifier']
                activity = activities_dict[column_name]
                j_old = columns_in.get_loc(column_name)
                e_in = entities_in[i][j_old]
                e_in_identifier = e_in['identifier']
                self.create_relation(self.GENERATION, a=e_out_identifier, b=activity)
                self.create_relation(self.DERIVATION, a=e_out_identifier, b=e_in_identifier)
                
                entities_out[i][j] = e_out
        
        # Rearrange unchanged columns:
        for col_name in columns_out:
            if col_name in columns_in:
                old_j = columns_in.get_loc(col_name)
                new_j = columns_out.get_loc(col_name)
                entities_out[:,new_j] = entities_in[:,old_j]
        
        # Update current values:
        self.set_current_values(df_out, entities_out)
        
        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))

        return self
    
    @timing
    def get_prov_value_transformation(self, df_out, value, function_name='Value Transformation'):
        """Return provenance document related to value transformation function.
        Used when a value inside the dataframe is replaced.
        
        Keyword argument:
        df_out -- the output dataframe
        value -- replaced value
        """
        self.initialize()
        # Get current values:
        entities_in = self.current_ent

        # Output values:
        columns_out = df_out.columns
        
        # Create value transformation activity:
        act_id = self.create_activity(function_name)
        
        for i in range(self.current_m):
            for j in range(self.current_n):
                value = str(df_out.iat[i, j])
                
                e_in = entities_in[i][j]
                e_in_identifier = e_in['identifier']
                record_id = e_in['attributes']['record_id']
                val_in = e_in['attributes']['value']
                
                # Check if the input value is the replaced value
                if str(val_in) != str(value):
                    # Create new entity with the new value
                    ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                    e_out = self.create_entity(ent_id, record_id, value, columns_out[j], self.instance)
                    e_out_identifier = e_out['identifier']
                    self.create_relation(self.GENERATION, a=e_out_identifier, b=act_id)
                    self.create_relation(self.USE, a=act_id, b=e_in_identifier)
                    self.create_relation(self.DERIVATION, a=e_out_identifier, b=e_in_identifier)
                    entities_in[i][j] = e_out

        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))
        # Update current values:
        self.set_current_values(df_out, entities_in)

        return self
    
    # TODO: imputation relativo alla singola colonna o a tutte insieme?
    #       Puo comprendere un sottoinsieme di colonne o tutte insieme?
    @timing
    def get_prov_imputation(self, df_out, isIndipendent=True, function_name='Imputation'):
        """Return provenance document related to imputation function."""
        self.initialize()
        # Get current values:
        entities_in = self.current_ent
        # Output values:
        columns_out = df_out.columns
        
        # Create a single imputation activity if the imputation is related to a single column:
        if isIndipendent:
            act_id = self.create_activity(function_name) 
            
        for j in range(self.current_n):
            if not isIndipendent:
                act_id = self.create_activity(function_name) 
            for i in range(self.current_m):
                value = str(df_out.iat[i, j])
                
                e_in = entities_in[i][j]
                e_in_identifier = e_in['identifier']
                record_id = e_in['attributes']['record_id']
                val_in = e_in['attributes']['value']
                
                if val_in == 'nan':
                    # Create new entity with the new value
                    ent_id = self.NAMESPACE_ENTITY + str(uuid.uuid4())
                    e_out = self.create_entity(ent_id, record_id, value, columns_out[j], self.instance)
                    e_out_identifier = e_out['identifier']
                    self.create_relation(self.GENERATION, a=e_out_identifier, b=act_id)
                    self.create_relation(self.USE, a=act_id, b=e_in_identifier)
                    self.create_relation(self.DERIVATION, a=e_out_identifier, b=e_in_identifier)
                    entities_in[i][j] = e_out
                    

        # Save provenance document in json file:
        self.save_json_prov(os.path.join(self.results_path, self.instance))
        # Update current values:
        self.set_current_values(df_out, entities_in)

        return self